In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import credentials
import requests
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame
#pulling data API
key =credentials.login['key_isaham']  # extract password

In [2]:
# filter by sector
#def sector_filter(sector):

  # pulling data from api
  #URL = 'https://admin.isaham.my/api/stocks/list'
  #r = requests.get(url = URL) 
  #data = r.json() 
  #df = pd.DataFrame(data['stocks'],columns = ['code','symbol','name','primary-sector','secondary-sector'])

  #wl = df[df["primary-sector"]==sector]
  #wl = list(wl.symbol)

  #return(wl)

In [3]:
#sector = "Construction"

# defining the password
#key = credentials.login['key_isaham']
#wl = sector_filter(sector)

### Validation STEP

In [4]:
# Define the instruments to download.
cons = ['EKOVEST',
        'FAJAR',
        'GADANG',
        'GAMUDA',
        'GKENT',
        'HOHUP',
        'IJM',
        'JAKS',
        'KERJAYA',
        'KIMLUN',
        'MGB',
        'MRCB',
        'MUHIBAH',
        'PEB',
        'PESONA',
        'TALIWRK',
        'WIDAD'
        ]

       
# looping through all stocks
# dictionary of dataframes
ss={}
d = {}
# listing all stocks
all_stocks = [] 
mk={}


for m,stock in enumerate (cons):
        
    print("----------------PROCESSING {}------------------".format(stock))
    #symbol = "UNISEM"  # which stock to pull
    symbol = stock    
    
    response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
    data = response.json()
    #print(data)
    plt.style.use('fivethirtyeight')

    %config InlineBackend.figure_format = 'retina'
    %matplotlib inline

    # makes plots bigger
    plt.rcParams["figure.figsize"] = (20,8)
    
    #Tuning OB and OS level
    ob_cross=85
    os_cross=35

    df= pd.DataFrame.from_dict(data)
    df= df[['c','d','h','l','o','v']]
    df['d'] = pd.to_datetime(df['d'], dayfirst= True )
    #identify MA50
    n=50
    ma50=[] 

    for i in range(len(df["c"])-(n-1)):
        ma = round(sum(df["c"][i:i+(n)])/n,4)
        #print(ma)
        ma50.append(ma)
    
    df["MA50"] = pd.Series(ma50)
    df["MA50"] = df["MA50"].shift(n-1)

    #filter uptrend price above ma50
    # Calculating the gap of both MA
    df["MAgap"] = df["c"] -  df["MA50"] 

    df.set_index('d',inplace=True,drop=True)
    df.index.name = None

    #create signal for uptrend price above ma50 
    #result not accurate for stochastic when just filter uptrend only
    df["Uptrend"]=df['MAgap']>0
    Uptrend=df["Uptrend"]
    Uptrend_signal=Uptrend[Uptrend==True]
    #signal stochastic
    #Create the "L14" column in the DataFrame
    df['L14'] = df['l'].rolling(window=14).min()

    #Create the "H14" column in the DataFrame
    df['H14'] = df['h'].rolling(window=14).max()

    #Create the "%K" column in the DataFrame
    df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

    #Create the "%D" column in the DataFrame
    df['%D'] = df['%K'].rolling(window=3).mean()

    #Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
    #A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
    df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

    #Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
    #A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
    df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 
    
    df['Stock_Name'] = symbol

    overbought =df['Sell OB']
    oversold =df['Buy OS']
    date_os=list(oversold[oversold == True].index)
    date_ob=list(overbought[overbought == True].index)
    #print(oversold[oversold == True])
    #print(df)
    mk["{}".format(stock)] = df
    # transaction recording
    buy = []
    sell = []
    sdate = []
    bdate = []
    wr = []
    pct = []
    vol = []
    ope =[]
    high = []
    low = []


    for n,i in enumerate(date_os):
        #print(n,"Buy:", i, "Price:", df['c'].loc[i])
        buy.append(df['c'].loc[i])
        vol.append(df['v'].loc[i])
        ope.append(df['o'].loc[i])
        high.append(df['h'].loc[i])
        low.append(df['l'].loc[i])
        bdate.append(i)
        r=[]
        #print("before", len(r))
        for m,k in enumerate(date_ob):
            
            if k>i:
                r.append(m)
                #print('after',len(r))
                if len(r) ==1:
                    #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                    #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                    sell.append(df['c'].loc[k])
                    sdate.append(k)
                    pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                    pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                elif len(r)>1:
                    pass
    diff =len(buy)-len(sell)
    # Checking for final transaction, make sure it is sell, as we want to close the position
    if len(buy) == len(sell):
        #print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
        #print("Position closed")
        pass
    
    elif len(buy) != len(sell):
        #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
        del(buy[-diff:])
        del(bdate[-diff:])
        del(ope[-diff:])
        del(high[-diff:])
        del(vol[-diff:])
        del(low[-diff:])
        #print("Position closed")
    dct = {
        "Buy_Date": bdate,
        "Buy": buy,
        "Sell": sell,
        "Sell_Date": sdate,
        "Open":ope,
        'Volume': vol,
        "Low": low,
        "high": high,
        "Pct_change": pct    
    }  
    trans = pd.DataFrame(dct)
    wr = []
    for i in range(len(trans)):
        if trans["Pct_change"][i] > 0:
            wr.append(1)
        else:
            wr.append(0)
    trans["wr"] = pd.Series(wr)
    
    Name_stock = []
    for s in range(len(trans)):
        Name_stock.append(symbol)
        
    #Signal_buy = []
    #for t in range(len(trans)):
        #Signal_buy.append(1)
        
    
    trans['Stock'] = pd.Series(Name_stock)
    #trans['Signal'] = pd.Series(Signal_buy)
    
    # scaling certain features only
    col_names = ['Buy', 'Low', 'Open', 'Volume','high']  
    features = trans[col_names]
    
    #features = dataf[col_names]
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
    # convert the array back to a dataframe
    dataset = DataFrame(features)
    
    
    col_notscale=['Buy_Date','Stock','wr']
    features_notscale=trans[col_notscale]
    
    # saving in multiple df
    d["{}".format(stock)] = dataset
    ss["{}".format(stock)] = features_notscale
    
    # list of all stocks
    all_stocks.append(d[stock])
    
    
    #print(trans)
    
    #dct_1={ 'Number of stock':dct
    
    #}
    #trans_1 = pd.DataFrame(dct_1)
    
    #print(trans[0:])
    
    #trans.to_csv('result {}.csv'.format(symbol), index=False)
    #C:\Users\Mo Syamil\Documents\data scientiest trial\strategy stho\result
            
    print("----------------DONE {}------------------".format(symbol))
    #print(trans)
    #concat_all= pd.concat([trans], axis=0)
    #print(concat_all)
    #data_1=[]
    
    #for d in  range(len(trans)):
        #concat_all=pd.concat([trans], axis=0)
        #data_1.append(list(trans))
        #print(data_1)
    #c_data= trans[m]
    #print(concat_all)
    #print(trans)
    
    
        
    #print(trans)
    #print()

    
# Merging all df
#concat_all= pd.concat([trans], axis=0)
#print(trans.append(m))
#print(trans)
#print(concat_all)

#print("Finish")

    



----------------PROCESSING EKOVEST------------------
----------------DONE EKOVEST------------------
----------------PROCESSING FAJAR------------------
----------------DONE FAJAR------------------
----------------PROCESSING GADANG------------------
----------------DONE GADANG------------------
----------------PROCESSING GAMUDA------------------
----------------DONE GAMUDA------------------
----------------PROCESSING GKENT------------------
----------------DONE GKENT------------------
----------------PROCESSING HOHUP------------------
----------------DONE HOHUP------------------
----------------PROCESSING IJM------------------
----------------DONE IJM------------------
----------------PROCESSING JAKS------------------
----------------DONE JAKS------------------
----------------PROCESSING KERJAYA------------------
----------------DONE KERJAYA------------------
----------------PROCESSING KIMLUN------------------
----------------DONE KIMLUN------------------
----------------PROCESSING MGB--

In [5]:
# combining all data generated
df_notscale = pd.concat(list(ss.values()),ignore_index=True)

In [6]:
df_notscale

,Buy_Date,Stock,wr
0,1994-09-02,EKOVEST,1
1,1994-09-26,EKOVEST,0
2,1994-10-11,EKOVEST,0
3,1995-12-20,EKOVEST,1
4,1995-12-26,EKOVEST,1
...,...,...,...
2004,2019-08-23,WIDAD,1
2005,2020-01-20,WIDAD,0
2006,2020-09-10,WIDAD,1
2007,2020-09-15,WIDAD,1


In [7]:
df_scale = pd.concat(list(d.values()),ignore_index=True)

In [8]:
df_scale=df_scale.rename(columns={0: 'Buy', 1: 'Low',2: 'Open', 3: 'Volume',4: 'High'})

In [9]:
df_scale

,Buy,Low,Open,Volume,High
0,1.351782,1.396746,1.341967,-0.268465,1.381228
1,1.527719,1.630000,1.570744,-0.268992,1.529451
2,1.603056,1.578321,1.545629,-0.256650,1.554303
3,0.233907,0.281687,0.248315,-0.256705,0.219846
4,0.171201,0.126184,0.146485,-0.248672,0.194994
...,...,...,...,...,...
2004,-0.059839,0.008805,-0.012188,-0.074369,-0.045371
2005,2.181964,2.243509,2.238297,1.462881,2.207774
2006,2.181964,2.202126,2.238297,0.576068,2.167540
2007,2.181964,2.243509,2.197379,0.094831,2.167540


In [10]:
#df_scale=df_scale[['Buy', 'Low', 'Open', 'Volume','High']]

In [11]:
result_scaler = pd.concat([df_notscale, df_scale], axis=1)

In [12]:
result_scaler

,Buy_Date,Stock,wr,Buy,Low,Open,Volume,High
0,1994-09-02,EKOVEST,1,1.351782,1.396746,1.341967,-0.268465,1.381228
1,1994-09-26,EKOVEST,0,1.527719,1.630000,1.570744,-0.268992,1.529451
2,1994-10-11,EKOVEST,0,1.603056,1.578321,1.545629,-0.256650,1.554303
3,1995-12-20,EKOVEST,1,0.233907,0.281687,0.248315,-0.256705,0.219846
4,1995-12-26,EKOVEST,1,0.171201,0.126184,0.146485,-0.248672,0.194994
...,...,...,...,...,...,...,...,...
2004,2019-08-23,WIDAD,1,-0.059839,0.008805,-0.012188,-0.074369,-0.045371
2005,2020-01-20,WIDAD,0,2.181964,2.243509,2.238297,1.462881,2.207774
2006,2020-09-10,WIDAD,1,2.181964,2.202126,2.238297,0.576068,2.167540
2007,2020-09-15,WIDAD,1,2.181964,2.243509,2.197379,0.094831,2.167540


In [13]:
df_all = result_scaler.sort_values(by="Buy_Date")
# setting Dates as index
df_all.set_index('Buy_Date', inplace=True)

In [14]:
# Encoding the stock names
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_all["Stock"] = le.fit_transform(df_all["Stock"])
df_all.head(20)

,Stock,wr,Buy,Low,Open,Volume,High
Buy_Date,,,,,,,
1994-05-10,11,1,1.560532,1.452491,1.424093,-0.273529,1.564705
1994-05-13,4,1,1.748854,1.749644,1.759906,-0.402418,1.733889
1994-05-13,11,1,1.621443,1.565091,1.535668,-0.354975,1.625148
1994-05-18,11,1,1.590987,1.540092,1.510896,-0.336287,1.594926
1994-05-23,4,1,1.653100,1.701202,1.687360,-0.501756,1.638450
1994-05-26,11,1,1.475195,1.502490,1.473636,-0.419755,1.474040
1994-06-08,4,1,1.653100,1.701202,1.687360,-0.501756,1.638450
1994-06-13,4,1,1.796670,1.773927,1.759906,-0.257136,1.781547
1994-06-30,3,1,-1.821050,-1.839962,-1.844523,-1.015560,-1.823129


In [15]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [16]:
# Separate input features and target
#X = df_all.drop(['wr','Pct_change','Sell_Date','Sell','Volume','Low','high'], axis=1)
X = df_all.drop(['wr'], axis=1)
y = df_all.wr

In [17]:
df_all["wr"].value_counts()

1    1195
0     814
Name: wr, dtype: int64

In [18]:
# setting up testing and training sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=0, shuffle=False)

In [19]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score


start_y = 17  
end_y = 21

test_prec = []
train_acc = []
test_acc = []
for i in range((end_y + 1) - start_y):  # years remaining
    for j in range(1,12,1):  # iteratting from 1 - 12
        
        # Fill the string with zeros until it is 2 characters long  
        date1 = "20{}-{}-01".format(start_y + i, str(j).zfill(2))  # starting of the test set
        date2 = "20{}-{}-01".format(start_y + i, str(j + 2).zfill(2))  # contain in 1 month only
        
        if j == 11:
            date2 = "20{}-{}-01".format(start_y + (i + 1), str((j + 2) - j).zfill(2))  # contain in 1 month only
        
        if date2 < str(df_all.index[-1]):  # limiting the splitting for duration below the last date
            
            print("Start:", date1)
            print("End:", date2)

            # splitting train test
            X_train, y_train = X[:date1], y[:date1]
            X_test, y_test = X[date1:date2], y[date1:date2]
            print("#### {}, {} ####".format(len(X_train), len(X_test)))
            
            

            # defining the model
            model = XGBClassifier(verbosity=0)
            # training the model
            model.fit(X_train, y_train)
            
            # predicting the train and test set
            # train
            y_pred = model.predict(X_train)
            pred1 = [round(value) for value in y_pred]
            train_accuracy = accuracy_score(y_train, pred1)
            train_acc.append(train_accuracy)
            
            # test
            y_pred = model.predict(X_test)
            pred2 = [round(value) for value in y_pred]
            test_accuracy = accuracy_score(y_test, pred2)
            test_acc.append(test_accuracy)
            print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
            
            precision = precision_score(y_test, pred2)
            test_prec.append(precision)
            #prec["{}".format(date1)] =  test_prec
            print("Test Precision: %.2f%%" % (precision * 100.0))
            
            if i == 0:
                print("----------------------------- DONE ROUND {} -----------------------------".format(i*10 + j))
            else:
                print("----------------------------- DONE ROUND {} ----------------------------- \n".format(i*10 + j+(2*i)))
# remove nan values for list test_acc
new_test_acc = [x for x in test_acc if np.isnan(x) == False]
avg_1 = sum(train_acc)/len(train_acc)  # calculating the average accuracy of the train set
avg_2 = round(sum(new_test_acc)/len(new_test_acc),4)  # calculating the average accuracy of the test set
print("Overall Train accuracy is: {}".format(avg_1))
print("Overall Test accuracy is: {}".format(avg_2)) 

avg_3 = sum(test_prec)/len(test_prec)  # calculating the average accuracy of the train set
print("Overall Test precision is: {}".format(avg_3))

            
            
            
            
            
            

Start: 2017-01-01
End: 2017-03-01
#### 1664, 22 ####


C:\Users\Public\Documents\anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Test Accuracy: 59.09%
Test Precision: 85.71%
----------------------------- DONE ROUND 1 -----------------------------
Start: 2017-02-01
End: 2017-04-01
#### 1678, 17 ####
Test Accuracy: 58.82%
Test Precision: 100.00%
----------------------------- DONE ROUND 2 -----------------------------
Start: 2017-03-01
End: 2017-05-01
#### 1686, 32 ####
Test Accuracy: 50.00%
Test Precision: 71.43%
----------------------------- DONE ROUND 3 -----------------------------
Start: 2017-04-01
End: 2017-06-01
#### 1695, 39 ####
Test Accuracy: 48.72%
Test Precision: 52.00%
----------------------------- DONE ROUND 4 -----------------------------
Start: 2017-05-01
End: 2017-07-01
#### 1717, 24 ####
Test Accuracy: 45.83%
Test Precision: 42.11%
----------------------------- DONE ROUND 5 -----------------------------
Start: 2017-06-01
End: 2017-08-01
#### 1734, 13 ####
Test Accuracy: 76.92%
Test Precision: 70.00%
----------------------------- DONE ROUND 6 -----------------------------
Start: 2017-07-01
End: 201

C:\Users\Public\Documents\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Public\Documents\anaconda\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\Public\Documents\anaconda\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Test Accuracy: nan%
Test Precision: 0.00%
----------------------------- DONE ROUND 39 ----------------------------- 

Start: 2020-04-01
End: 2020-06-01
#### 1938, 3 ####
Test Accuracy: 66.67%
Test Precision: 100.00%
----------------------------- DONE ROUND 40 ----------------------------- 

Start: 2020-05-01
End: 2020-07-01
#### 1938, 20 ####
Test Accuracy: 65.00%
Test Precision: 61.54%
----------------------------- DONE ROUND 41 ----------------------------- 

Start: 2020-06-01
End: 2020-08-01
#### 1941, 23 ####
Test Accuracy: 73.91%
Test Precision: 73.33%
----------------------------- DONE ROUND 42 ----------------------------- 

Start: 2020-07-01
End: 2020-09-01
#### 1958, 11 ####
Test Accuracy: 81.82%
Test Precision: 85.71%
----------------------------- DONE ROUND 43 ----------------------------- 

Start: 2020-08-01
End: 2020-10-01
#### 1964, 17 ####
Test Accuracy: 70.59%
Test Precision: 80.00%
----------------------------- DONE ROUND 44 ----------------------------- 

Start: 2020-


#Overall Train accuracy is: 0.9767433218882797
#Overall Test accuracy is: 0.5841
#Overall Test precision is: 0.621037973277676

### Prediction STEP

In [22]:


# Define the instruments to download.
cons = ['EKOVEST',
        'FAJAR',
        'GADANG',
        'GAMUDA',
        'GKENT',
        'HOHUP',
        'IJM',
        'JAKS',
        'KERJAYA',
        'KIMLUN',
        'MGB',
        'MRCB',
        'MUHIBAH',
        'PEB',
        'PESONA',
        'PTARAS',
        'TALIWRK',
        'WIDAD',
        'ZECON'
        ]        
# looping through all stocks

# Scale dictionary of dataframes for training set(scaling by stock)
ss={}
d = {}

# Scale dictionary of dataframes for prediction set(scaling by stock)
sp={}
nsp={}

for m,stock in enumerate (cons):
        
    print("----------------PROCESSING {}------------------".format(stock))
    symbol = stock    # Which stock to pull    
    response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
    data = response.json()
    
    # Tuning OB and OS level
    ob_cross=85
    os_cross=35

    df= pd.DataFrame.from_dict(data)
    df= df[['c','d','h','l','o','v']]
    df['d'] = pd.to_datetime(df['d'], dayfirst= True )
    
    # Identify MA50
    n=50
    ma50=[] 

    for i in range(len(df["c"])-(n-1)):
        ma = round(sum(df["c"][i:i+(n)])/n,4)
        ma50.append(ma)
    
    df["MA50"] = pd.Series(ma50)
    df["MA50"] = df["MA50"].shift(n-1)

    # Filter uptrend price above ma50
    # Calculating the gap of both MA
    df["MAgap"] = df["c"] -  df["MA50"] 
    
    # Set date as index
    df.set_index('d',inplace=True,drop=False)
    df.index.name = None

    # Create signal for uptrend price above ma50 
    # Result not accurate for stochastic when just filter uptrend only
    df["Uptrend"]=df['MAgap']>0
    Uptrend=df["Uptrend"]
    Uptrend_signal=Uptrend[Uptrend==True]
    
    # Signal stochastic
    # Create the "L14" column in the DataFrame
    df['L14'] = df['l'].rolling(window=14).min()

    # Create the "H14" column in the DataFrame
    df['H14'] = df['h'].rolling(window=14).max()

    # Create the "%K" column in the DataFrame
    df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

    # Create the "%D" column in the DataFrame
    df['%D'] = df['%K'].rolling(window=3).mean()

    # Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
    # A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
    df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

    # Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
    # A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
    df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 
    
    # Create a column in the DataFrame for name of stock
    df['Stock_Name'] = symbol

    # Define overbought and oversold
    overbought =df['Sell OB']
    oversold =df['Buy OS']
    
    # Use index(date) for create strategy using overbought and oversold
    date_os=list(oversold[oversold == True].index)
    date_ob=list(overbought[overbought == True].index)
    
    # Scaling certain features only
    col_names_pred = ['c', 'l', 'o', 'v','h']  
    features_pred = df[col_names_pred]
    
    # Scaling for prediction data
    scaler_pred = StandardScaler().fit(features_pred.values)
    features_pred = scaler_pred.transform(features_pred.values)
    
    # Convert the array back to a dataframe
    dataset_pred = DataFrame(features_pred)
    
    # Prepare prediction data that no need to scale
    col_notscale_pred=['d','Stock_Name','Buy OS']
    features_notscale_pred=df[col_notscale_pred]
    
    # Saving in multiple variable
    sp["{}".format(stock)] = dataset_pred
    nsp["{}".format(stock)] = features_notscale_pred
    
    # Transaction recording
    buy = []
    sell = []
    sdate = []
    bdate = []
    wr = []
    pct = []
    vol = []
    ope =[]
    high = []
    low = []

    # Record close,open,vol,high,low for buy transaction
    for n,i in enumerate(date_os):
        buy.append(df['c'].loc[i])
        vol.append(df['v'].loc[i])
        ope.append(df['o'].loc[i])
        high.append(df['h'].loc[i])
        low.append(df['l'].loc[i])
        bdate.append(i)
        r=[]
        
        for m,k in enumerate(date_ob):
            
            if k>i:
                r.append(m)
                if len(r) ==1:
                    # Record close,percentage change for sell transaction
                    sell.append(df['c'].loc[k])
                    sdate.append(k)
                    pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                    pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                elif len(r)>1:
                    pass
    diff =len(buy)-len(sell)
    # Checking for final transaction, make sure it is sell, as we want to close the position
    if len(buy) == len(sell):
        pass   
    elif len(buy) != len(sell):
        # Deleting excess buy transaction after we close position
        del(buy[-diff:])
        del(bdate[-diff:])
        del(ope[-diff:])
        del(high[-diff:])
        del(vol[-diff:])
        del(low[-diff:])
    dct = {
        "Buy_Date": bdate,
        "Buy": buy,
        "Sell": sell,
        "Sell_Date": sdate,
        "Open":ope,
        'Volume': vol,
        "Low": low,
        "high": high,
        "Pct_change": pct    
    }  
    # Save document for stochastic strategy
    trans = pd.DataFrame(dct)
    
    # Define variable for winning rate
    wr = []
    for i in range(len(trans)):
        if trans["Pct_change"][i] > 0:
            wr.append(1)
        else:
            wr.append(0)
    
    # Define variable for name of stock
    Name_stock = []
    for s in range(len(trans)):
        Name_stock.append(symbol)
        
    # Add new column
    trans["wr"] = pd.Series(wr)
    trans['Stock'] = pd.Series(Name_stock)
    
    # Scaling certain features only for training data
    col_names = ['Buy', 'Low', 'Open', 'Volume','high']  
    features = trans[col_names]
    
    # Prepare prediction data that no need to scale
    scaler = StandardScaler().fit(features.values)
    features = scaler.transform(features.values)
    
    # Convert the array back to a dataframe
    dataset = DataFrame(features)    
    col_notscale = ['Buy_Date','Stock','wr']
    features_notscale = trans[col_notscale]
    
    # Saving in multiple variable
    d["{}".format(stock)] = dataset
    ss["{}".format(stock)] = features_notscale
    print("----------------DONE {}------------------".format(symbol))

# Combining all training data generated
df_notscale = pd.concat(list(ss.values()),ignore_index=True)
df_scale = pd.concat(list(d.values()),ignore_index=True)
df_scale=df_scale.rename(columns={0: 'Buy', 1: 'Low',2: 'Open', 3: 'Volume',4: 'High'})
result_scaler = pd.concat([df_notscale, df_scale], axis=1)

# Setting Dates as index
df_all = result_scaler.sort_values(by="Buy_Date")
df_all.set_index('Buy_Date', inplace=True)

# Encoding the stock names
le = LabelEncoder()
df_all["Stock"] = le.fit_transform(df_all["Stock"])

# Define the splitting date (for training we use 2 years data)
# Splitting train test
#day = dt.datetime.today().day
#month = dt.datetime.today().month
#year = dt.datetime.today().year - 2
#date1 = "{}-{}-01".format(year, str(month).zfill(2), str(day).zfill(2))
#date2 = str(dt.datetime.today().date() - dt.timedelta(days=1))  # until yesterday only

# Separate input features and target
X = df_all.drop(['wr'], axis=1)
y = df_all.wr



----------------PROCESSING EKOVEST------------------
----------------DONE EKOVEST------------------
----------------PROCESSING FAJAR------------------
----------------DONE FAJAR------------------
----------------PROCESSING GADANG------------------
----------------DONE GADANG------------------
----------------PROCESSING GAMUDA------------------
----------------DONE GAMUDA------------------
----------------PROCESSING GKENT------------------
----------------DONE GKENT------------------
----------------PROCESSING HOHUP------------------
----------------DONE HOHUP------------------
----------------PROCESSING IJM------------------
----------------DONE IJM------------------
----------------PROCESSING JAKS------------------
----------------DONE JAKS------------------
----------------PROCESSING KERJAYA------------------
----------------DONE KERJAYA------------------
----------------PROCESSING KIMLUN------------------
----------------DONE KIMLUN------------------
----------------PROCESSING MGB--

In [23]:
# Splitting train test
# train data (from listing IPO until 1 jan 2021)
X_train, y_train = X['01-01-2016': '01-01-2021'], y['01-01-2016':'01-01-2021']

In [27]:
X_train

,Stock,Buy,Low,Open,Volume,High
Buy_Date,,,,,,
2016-01-05,9,-0.046301,-0.266067,-0.262813,0.361699,-0.013759
2016-01-05,18,-0.181689,-0.196937,-0.177559,-0.233245,-0.195016
2016-01-12,7,1.099969,1.145555,1.102719,-0.596792,1.117141
2016-01-15,17,-0.793520,-0.818863,-0.830547,-0.377733,-0.809831
2016-01-21,18,-0.191996,-0.142798,-0.156272,-0.439156,-0.205267
...,...,...,...,...,...,...
2020-12-23,12,-0.697103,-0.683614,-0.686512,0.409287,-0.689518
2020-12-23,11,-1.007714,-1.007366,-1.013347,-0.200157,-1.013998
2020-12-23,2,-0.429861,-0.430190,-0.430676,0.797749,-0.425331


In [28]:
y_train

Buy_Date
2016-01-05    1
2016-01-05    0
2016-01-12    0
2016-01-15    0
2016-01-21    0
             ..
2020-12-23    1
2020-12-23    0
2020-12-23    1
2020-12-23    0
2020-12-29    0
Name: wr, Length: 515, dtype: int64

In [31]:
# Combining all prediction data generated
df_notscale_pred = pd.concat(list(nsp.values()),ignore_index=True)
df_scale_pred = pd.concat(list(sp.values()),ignore_index=True)

In [32]:
# Rename column
df_scale_pred = df_scale_pred.rename(columns={0: 'Buy', 1: 'Low',2: 'Open', 3: 'Volume',4: 'High'})

In [33]:
# Concat all
result_scaler_pred = pd.concat([df_notscale_pred, df_scale_pred], axis=1)
result_scaler_pred = result_scaler_pred.rename(columns={'d': 'Date'})

# Setting Dates as index
df_all_pred = result_scaler_pred.sort_values(by="Date")
df_all_pred.set_index('Date', inplace=True)

# Encoding the stock names
le = LabelEncoder()
df_all_pred["Stock"] = le.fit_transform(df_all_pred["Stock_Name"])

In [36]:
X_trainss, y_train = X_train.to_numpy(), y_train.to_numpy()

In [37]:
# Training the model
upsampled = XGBClassifier(verbosity=0,random_seed=0).fit(X_trainss, y_train)

In [38]:
latest = df_all_pred['01-01-2021':]

In [45]:

dct_m = {}
symbol =[]
predicted=[]
date=[]

for m in range(len(latest)):
    if latest["Buy OS"].iloc[m] == True:
        # predicting the train and test set
        inputs = latest.drop(['Stock_Name','Buy OS'] ,axis=1).iloc[m].values
        
        #print(inputs)
        inputs = inputs.reshape(1,-1)
        y_pred = upsampled.predict(inputs)
        pred1 = [round(value) for value in y_pred]
        #print(pred1)
        dct_m["symbol"] = latest["Stock_Name"].iloc[m]
        symbol.append(dct_m["symbol"])
        
        dct_m["predicted_class"] =  pred1[0]
        predicted.append(dct_m["predicted_class"])
        
        dct_m['date_uploaded'] = latest.index[m]
        date.append(dct_m['date_uploaded'])
        print(dct_m)
    else:
        dct_m["symbol"] = latest["Stock_Name"].iloc[m]
        symbol.append(dct_m["symbol"])
        dct_m["predicted_class"] = "No buy signal. No prediction"
        predicted.append(dct_m["predicted_class"])
        
        dct_m['date_uploaded'] = latest.index[m]
        date.append(dct_m['date_uploaded'])
        #print("None of the stock in WL shows buy signal.\n No prediction")
        pass
        print(dct_m)
    mongo_db = {
        'Date Predicted':date,
        'Stock' :symbol,
        "Predicted_Class" :predicted,
        
    }
    Mongo_db = pd.DataFrame(mongo_db)

{'symbol': 'EKOVEST', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'IJM', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'GADANG', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'GAMUDA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'GKENT', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'WIDAD', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'JAKS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'KIMLUN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-04 00:00:00')}
{'symbol': 'ZECON', 'p

{'symbol': 'HOHUP', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-13 00:00:00')}
{'symbol': 'GADANG', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-13 00:00:00')}
{'symbol': 'MGB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-13 00:00:00')}
{'symbol': 'MRCB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-13 00:00:00')}
{'symbol': 'ZECON', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-13 00:00:00')}
{'symbol': 'EKOVEST', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-13 00:00:00')}
{'symbol': 'KERJAYA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-14 00:00:00')}
{'symbol': 'PTARAS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-14 00:00:00')}
{'symbol': 'HOHUP', '

{'symbol': 'PESONA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-25 00:00:00')}
{'symbol': 'EKOVEST', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-25 00:00:00')}
{'symbol': 'MGB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-25 00:00:00')}
{'symbol': 'MRCB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-25 00:00:00')}
{'symbol': 'IJM', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-25 00:00:00')}
{'symbol': 'HOHUP', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-25 00:00:00')}
{'symbol': 'PEB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-25 00:00:00')}
{'symbol': 'WIDAD', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-01-25 00:00:00')}
{'symbol': 'FAJAR', 'predict

{'symbol': 'GADANG', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-02-11 00:00:00')}
{'symbol': 'KIMLUN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-02-11 00:00:00')}
{'symbol': 'HOHUP', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-02-11 00:00:00')}
{'symbol': 'GAMUDA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-02-15 00:00:00')}
{'symbol': 'PESONA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-02-15 00:00:00')}
{'symbol': 'MRCB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-02-15 00:00:00')}
{'symbol': 'JAKS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-02-15 00:00:00')}
{'symbol': 'FAJAR', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-02-15 00:00:00')}
{'symbol': 'TALIWRK', 

{'symbol': 'PTARAS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-01 00:00:00')}
{'symbol': 'MUHIBAH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-01 00:00:00')}
{'symbol': 'ZECON', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-01 00:00:00')}
{'symbol': 'JAKS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-01 00:00:00')}
{'symbol': 'IJM', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-01 00:00:00')}
{'symbol': 'MRCB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-01 00:00:00')}
{'symbol': 'KIMLUN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-01 00:00:00')}
{'symbol': 'GKENT', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-01 00:00:00')}
{'symbol': 'HOHUP', 'pre

...

{'symbol': 'KERJAYA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-26 00:00:00')}
{'symbol': 'PESONA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-26 00:00:00')}
{'symbol': 'MRCB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-26 00:00:00')}
{'symbol': 'HOHUP', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-26 00:00:00')}
{'symbol': 'WIDAD', 'predicted_class': 1, 'date_uploaded': Timestamp('2021-03-26 00:00:00')}
{'symbol': 'MGB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-26 00:00:00')}
{'symbol': 'PTARAS', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-26 00:00:00')}
{'symbol': 'GAMUDA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-03-26 00:00:00')}
{'symbol': 'TALIWRK', 'predicted_class': 'No buy si

{'symbol': 'MRCB', 'predicted_class': 1, 'date_uploaded': Timestamp('2021-04-01 00:00:00')}
{'symbol': 'MGB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-01 00:00:00')}
{'symbol': 'KERJAYA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-01 00:00:00')}
{'symbol': 'FAJAR', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-01 00:00:00')}
{'symbol': 'KIMLUN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-02 00:00:00')}
{'symbol': 'MUHIBAH', 'predicted_class': 1, 'date_uploaded': Timestamp('2021-04-02 00:00:00')}
{'symbol': 'PEB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-02 00:00:00')}
{'symbol': 'GKENT', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-02 00:00:00')}
{'symbol': 'IJM', 'predicted_class': 'No buy signal. No prediction', 'date_uploade

{'symbol': 'ZECON', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-07 00:00:00')}
{'symbol': 'HOHUP', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-07 00:00:00')}
{'symbol': 'MUHIBAH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-07 00:00:00')}
{'symbol': 'EKOVEST', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-07 00:00:00')}
{'symbol': 'KERJAYA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-07 00:00:00')}
{'symbol': 'MGB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-07 00:00:00')}
{'symbol': 'IJM', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-07 00:00:00')}
{'symbol': 'GKENT', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-07 00:00:00')}
{'symbol': 'PTARAS', '

{'symbol': 'KIMLUN', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-13 00:00:00')}
{'symbol': 'TALIWRK', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-13 00:00:00')}
{'symbol': 'EKOVEST', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-13 00:00:00')}
{'symbol': 'WIDAD', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-13 00:00:00')}
{'symbol': 'MUHIBAH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-13 00:00:00')}
{'symbol': 'GAMUDA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-14 00:00:00')}
{'symbol': 'GADANG', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-14 00:00:00')}
{'symbol': 'HOHUP', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-14 00:00:00')}
{'symbol': 'JAK

{'symbol': 'GADANG', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-19 00:00:00')}
{'symbol': 'EKOVEST', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-19 00:00:00')}
{'symbol': 'IJM', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-19 00:00:00')}
{'symbol': 'GKENT', 'predicted_class': 1, 'date_uploaded': Timestamp('2021-04-19 00:00:00')}
{'symbol': 'MUHIBAH', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-19 00:00:00')}
{'symbol': 'KERJAYA', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-19 00:00:00')}
{'symbol': 'MRCB', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-19 00:00:00')}
{'symbol': 'TALIWRK', 'predicted_class': 'No buy signal. No prediction', 'date_uploaded': Timestamp('2021-04-19 00:00:00')}
{'symbol': 'PEB', 'predicted_class': 'No buy si

In [46]:
Mongo_db.set_index('Date Predicted', inplace=True)
Mongo_db.index.name = None


In [51]:
Mongo_db[Mongo_db.Predicted_Class==1]

,Stock,Predicted_Class
2021-01-05,HOHUP,1
2021-01-07,IJM,1
2021-01-08,KERJAYA,1
2021-01-12,JAKS,1
2021-01-20,FAJAR,1
2021-03-03,MGB,1
2021-03-05,MGB,1
2021-03-08,KERJAYA,1
2021-03-10,PTARAS,1
2021-03-12,KERJAYA,1


In [41]:
Mongo_db[Mongo_db.Predicted_Class==0]

,Stock,Predicted_Class
2021-01-05,TALIWRK,0
2021-02-26,TALIWRK,0
2021-03-03,JAKS,0
2021-03-08,JAKS,0
2021-03-16,TALIWRK,0
2021-03-22,TALIWRK,0
2021-03-25,HOHUP,0
2021-04-01,HOHUP,0
2021-04-05,WIDAD,0
2021-04-07,WIDAD,0


### CHECK result backtest

In [53]:
# Define the instruments to download.
#tech = ["GREATEC", 
       #"INARI"]

cons = ['EKOVEST',
        'FAJAR',
        'GADANG',
        'GAMUDA',
        'GKENT',
        'HOHUP',
        'IJM',
        'JAKS',
        'KERJAYA',
        'KIMLUN',
        'MGB',
        'MRCB',
        'MUHIBAH',
        'PEB',
        'PESONA',
        'PTARAS',
        'TALIWRK',
        'WIDAD',
        'ZECON'
        ]        
       
# looping through all stocks
# dictionary of dataframes
d = {}
# listing all stocks
all_stocks = []       


for m,stock in enumerate (cons):
        
    print("----------------PROCESSING {}------------------".format(stock))
    #symbol = "UNISEM"  # which stock to pull
    symbol = stock    
    
    response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
    data = response.json()
    #print(data)
    plt.style.use('fivethirtyeight')

    %config InlineBackend.figure_format = 'retina'
    %matplotlib inline

    # makes plots bigger
    plt.rcParams["figure.figsize"] = (20,8)
    
    #Tuning OB and OS level
    ob_cross=85
    os_cross=35

    df= pd.DataFrame.from_dict(data)
    df= df[['c','d','h','l','o','v']]
    df['d'] = pd.to_datetime(df['d'], dayfirst= True )
    #identify MA50
    n=50
    ma50=[] 

    for i in range(len(df["c"])-(n-1)):
        ma = round(sum(df["c"][i:i+(n)])/n,4)
        #print(ma)
        ma50.append(ma)
    
    df["MA50"] = pd.Series(ma50)
    df["MA50"] = df["MA50"].shift(n-1)

    #filter uptrend price above ma50
    # Calculating the gap of both MA
    df["MAgap"] = df["c"] -  df["MA50"] 

    df.set_index('d',inplace=True,drop=True)
    df.index.name = None

    #create signal for uptrend price above ma50 
    #result not accurate for stochastic when just filter uptrend only
    df["Uptrend"]=df['MAgap']>0
    Uptrend=df["Uptrend"]
    Uptrend_signal=Uptrend[Uptrend==True]
    #signal stochastic
    #Create the "L14" column in the DataFrame
    df['L14'] = df['l'].rolling(window=14).min()

    #Create the "H14" column in the DataFrame
    df['H14'] = df['h'].rolling(window=14).max()

    #Create the "%K" column in the DataFrame
    df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

    #Create the "%D" column in the DataFrame
    df['%D'] = df['%K'].rolling(window=3).mean()

    #Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
    #A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
    df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

    #Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
    #A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
    df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 

    overbought =df['Sell OB']
    oversold =df['Buy OS']
    date_os=list(oversold[oversold == True].index)
    date_ob=list(overbought[overbought == True].index)
    
    # transaction recording
    buy = []
    sell = []
    sdate = []
    bdate = []
    wr = []
    pct = []
    vol = []
    ope =[]
    high = []
    low = []


    for n,i in enumerate(date_os):
        #print(n,"Buy:", i, "Price:", df['c'].loc[i])
        buy.append(df['c'].loc[i])
        vol.append(df['v'].loc[i])
        ope.append(df['o'].loc[i])
        high.append(df['h'].loc[i])
        low.append(df['l'].loc[i])
        bdate.append(i)
        r=[]
        #print("before", len(r))
        for m,k in enumerate(date_ob):
            
            if k>i:
                r.append(m)
                #print('after',len(r))
                if len(r) ==1:
                    #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                    #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                    sell.append(df['c'].loc[k])
                    sdate.append(k)
                    pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                    pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                elif len(r)>1:
                    pass
    diff =len(buy)-len(sell)
    # Checking for final transaction, make sure it is sell, as we want to close the position
    if len(buy) == len(sell):
        #print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
        #print("Position closed")
        pass
    
    elif len(buy) != len(sell):
        #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
        del(buy[-diff:])
        del(bdate[-diff:])
        del(ope[-diff:])
        del(high[-diff:])
        del(vol[-diff:])
        del(low[-diff:])
        #print("Position closed")
    dct = {
        "Buy_Date": bdate,
        "Buy": buy,
        "Sell": sell,
        "Sell_Date": sdate,
        "Open":ope,
        'Volume': vol,
        "Low": low,
        "high": high,
        "Pct_change": pct    
    }  
    trans = pd.DataFrame(dct)
    wr = []
    for i in range(len(trans)):
        if trans["Pct_change"][i] > 0:
            wr.append(1)
        else:
            wr.append(0)
    trans["wr"] = pd.Series(wr)
    
    Name_stock = []
    for s in range(len(trans)):
        Name_stock.append(symbol)
    
    Signal_buy = []
    for t in range(len(trans)):
        Signal_buy.append(1)
        
    
    trans['Stock'] = pd.Series(Name_stock)
    trans['Signal'] = pd.Series(Signal_buy)
    
    
    
    # saving in multiple df
    d["{}".format(stock)] = trans
    
    # list of all stocks
    all_stocks.append(d[stock])
    
    
    #print(trans)
    
    #dct_1={ 'Number of stock':dct
    
    #}
    #trans_1 = pd.DataFrame(dct_1)
    
    #print(trans[0:])
    
    #trans.to_csv('result {}.csv'.format(symbol), index=False)
    #C:\Users\Mo Syamil\Documents\data scientiest trial\strategy stho\result
            
    print("----------------DONE {}------------------".format(symbol))


----------------PROCESSING EKOVEST------------------
----------------DONE EKOVEST------------------
----------------PROCESSING FAJAR------------------
----------------DONE FAJAR------------------
----------------PROCESSING GADANG------------------
----------------DONE GADANG------------------
----------------PROCESSING GAMUDA------------------
----------------DONE GAMUDA------------------
----------------PROCESSING GKENT------------------
----------------DONE GKENT------------------
----------------PROCESSING HOHUP------------------
----------------DONE HOHUP------------------
----------------PROCESSING IJM------------------
----------------DONE IJM------------------
----------------PROCESSING JAKS------------------
----------------DONE JAKS------------------
----------------PROCESSING KERJAYA------------------
----------------DONE KERJAYA------------------
----------------PROCESSING KIMLUN------------------
----------------DONE KIMLUN------------------
----------------PROCESSING MGB--

In [54]:
df_all=pd.concat(list(d.values()),ignore_index=True)

In [55]:
# sorting all values by dates
df_all = df_all.sort_values(by="Buy_Date")

# setting Dates as index
df_all.set_index('Buy_Date', inplace=True)

In [56]:
df_all['01-01-2021':]

,Buy,Sell,Sell_Date,Open,Volume,Low,high,Pct_change,wr,Stock,Signal
Buy_Date,,,,,,,,,,,
2021-01-05,0.440,0.455,2021-04-08,0.450,940000.0,0.435,0.45,3.409091,1,HOHUP,1
2021-01-07,1.640,1.630,2021-03-03,1.640,6066000.0,1.590,1.65,-0.609756,0,IJM,1
2021-01-08,1.010,1.330,2021-03-29,1.010,1560500.0,0.990,1.01,31.683168,1,KERJAYA,1
2021-01-20,0.490,0.505,2021-02-08,0.490,269200.0,0.480,0.49,3.061224,1,FAJAR,1
2021-03-03,0.875,1.030,2021-03-16,0.880,417200.0,0.860,0.88,17.714286,1,MGB,1
2021-03-05,0.855,1.030,2021-03-16,0.850,145100.0,0.835,0.86,20.467836,1,MGB,1
2021-03-08,1.120,1.330,2021-03-29,1.090,1404700.0,1.090,1.15,18.750000,1,KERJAYA,1
2021-03-12,1.120,1.330,2021-03-29,1.130,1140600.0,1.120,1.15,18.750000,1,KERJAYA,1
2021-03-25,0.440,0.455,2021-04-08,0.425,1409500.0,0.425,0.44,3.409091,1,HOHUP,1
